# Model training 
En este notebook vamos a trabajar con los datos del notebook **00-Datos.ipynb** y el objetivo será entrenar  modelos sobre el set de entrenamiento obtenido para luego evaluar sus resultados.

In [ ]:
### Imports Python
import sys
import sklearn
import numpy as np
import os
import matplotlib as mpl
import matplotlib.pyplot as plt

### Imports data
import os
import tarfile
import urllib.request
import pandas as pd
import pickle


In [ ]:
## Importo la data
housing = pd.read_pickle("datasets/housing_4train.pkl")  

### Saco los valores que quiero predecir
housing_value = housing["median_house_value"].copy()
housing = housing.drop("median_house_value", axis=1) # drop labels for training set


In [ ]:
housing.columns

In [ ]:
X_test.columns

## Data Transformer

Una vez que ya vimos cómo es la estructura de nuestros datos y conocemos su comportamiento, nos dedicamos a transformarlos para que los algoritmos de ML que utilicemos funcionen mejor. En el caso de las variables categóricas esto consiste en generar un _ordinal enconder_ o un _OneHotEncoder_. En el caso de las variables numéricas vamos a querer que éstas tomen valores del mismo orden de magnitud y que tengan el mismo dominio. Para ésto transformamos los datos con un escaleo, en general un escaleo _normal_.

### Numéricos

In [ ]:
from sklearn.preprocessing import StandardScaler

### transformacion NORMAL de  los datos
housing_num = housing.drop(["ocean_proximity"],axis = 1) ### Trabajaemos con los datos numericos, dropeamos los categoricos

scaler = StandardScaler() ## Instancia de clase StandardScaler
scaler.fit(housing_num) ### el scaler necesita saber el valor medio y desvío de cada variable, el método fit los obtiene

housing_num_tr = scaler.transform(housing_num)

In [ ]:
housing_num_tr

El resultado es un numpy array con los valores normalizados.

### Cat

In [ ]:
from sklearn.preprocessing import OneHotEncoder

housing_cat = housing[["ocean_proximity"]]
cat_encoder = OneHotEncoder()
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
housing_cat_tr = housing_cat_1hot.toarray()


In [ ]:
cat_encoder.categories_[0]

## Tuti junti

In [ ]:
### juntamos los resultadsos numericos y los categoricos
data_full_prepared = np.concatenate((housing_num_tr, housing_cat_tr), axis=1)
features = list(housing_num.columns) + list(cat_encoder.categories_[0])

In [ ]:
len(features)

# Modelos

## Regresión Lineal

En la regresión lineal voy a sacar los Features caegóricos porque meten mucho ruido.

In [ ]:
from sklearn.linear_model import LinearRegression

X = housing_num_tr
Y = housing_value
lin_reg = LinearRegression() ### instancia de la clase
lin_reg.fit(X, Y) ### Fit
print("Los coeficientes de la regresión son:\n",lin_reg.coef_,lin_reg.intercept_)


### Ploteo algunos de los resultados

In [ ]:
y_pred = lin_reg.predict(X)

In [ ]:
fig, ax = plt.subplots(figsize = (24,7))
plt.subplot(121)
idx = 7
#y_pred =[y for _,y in sorted(zip(X.T[idx],y_pred))]
plt.scatter(X.T[idx],y_pred,alpha =0.5,label = "data predicted")
plt.scatter(X.T[idx],Y,alpha =0.5,label = "data")

plt.plot(X.T[idx],lin_reg.coef_[idx]*X.T[idx] + lin_reg.intercept_,color = "red",label = "lineal model")
plt.xlabel(features[idx],fontsize= 16)
plt.ylabel("housing_value",fontsize = 16)
plt.yticks(fontsize = 14);
plt.xticks(fontsize = 14);
plt.legend(fontsize = 14)

plt.subplot(122)
idx = 2
#y_pred =[y for _,y in sorted(zip(X.T[idx],y_pred))]
plt.scatter(X.T[idx],y_pred,alpha =0.5,label = "data predicted")
plt.scatter(X.T[idx],Y,alpha =0.5,label = "data")

plt.plot(X.T[idx],lin_reg.coef_[idx]*X.T[idx] + lin_reg.intercept_,color = "red",label = "lineal model")
plt.xlabel(features[idx],fontsize= 16)
plt.ylabel("housing_value",fontsize = 16)
plt.yticks(fontsize = 14);
plt.xticks(fontsize = 14);
plt.legend(fontsize = 14)

## DecisionTreeRegressor

Los árboles de decisión son capaces de definir un valor de la variable dependiente **Y**, para diferentes interavalos y condiciones de las variables independientes **X**. En este sentido este algoritmo es capaz de hacer una regresión.

In [ ]:
from sklearn.tree import DecisionTreeRegressor
X =  data_full_prepared
tree_reg = DecisionTreeRegressor(random_state=42)
tree_reg.fit(data_full_prepared, housing_value)


In [ ]:
y_pred = tree_reg.predict(data_full_prepared)

In [ ]:
fig, ax = plt.subplots(figsize = (24,7))
plt.subplot(121)
idx = 7
#y_pred =[y for _,y in sorted(zip(X.T[idx],y_pred))]
plt.scatter(X.T[idx],y_pred,alpha =0.5,label = "data predicted")
plt.scatter(X.T[idx],Y,alpha =0.5,label = "data")

plt.xlabel(features[idx],fontsize= 16)
plt.ylabel("housing_value",fontsize = 16)
plt.yticks(fontsize = 14);
plt.xticks(fontsize = 14);
plt.legend(fontsize = 14)

plt.subplot(122)
idx = 2
#y_pred =[y for _,y in sorted(zip(X.T[idx],y_pred))]
plt.scatter(X.T[idx],y_pred,alpha =0.5,label = "data predicted")
plt.scatter(X.T[idx],Y,alpha =0.5,label = "data")

plt.xlabel(features[idx],fontsize= 16)
plt.ylabel("housing_value",fontsize = 16)
plt.yticks(fontsize = 14);
plt.xticks(fontsize = 14);
plt.legend(fontsize = 14)

**LOS PUNTOS PREDICHOS SON IGUALES A LOS ORIGINALES**, ALGO HUELE MAL.

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(n_estimators=10, random_state=42)
forest_reg.fit(data_full_prepared, housing_value)
y_pred = forest_reg.predict(data_full_prepared)

In [ ]:
fig, ax = plt.subplots(figsize = (24,7))
plt.subplot(121)
idx = 7
#y_pred =[y for _,y in sorted(zip(X.T[idx],y_pred))]
plt.scatter(X.T[idx],y_pred,alpha =0.5,label = "data predicted")
plt.scatter(X.T[idx],Y,alpha =0.5,label = "data")

plt.xlabel(features[idx],fontsize= 16)
plt.ylabel("housing_value",fontsize = 16)
plt.yticks(fontsize = 14);
plt.xticks(fontsize = 14);
plt.legend(fontsize = 14)

plt.subplot(122)
idx = 2
#y_pred =[y for _,y in sorted(zip(X.T[idx],y_pred))]
plt.scatter(X.T[idx],y_pred,alpha =0.5,label = "data predicted")
plt.scatter(X.T[idx],Y,alpha =0.5,label = "data")

plt.xlabel(features[idx],fontsize= 16)
plt.ylabel("housing_value",fontsize = 16)
plt.yticks(fontsize = 14);
plt.xticks(fontsize = 14);
plt.legend(fontsize = 14)

# Métricas en una regresión

Nos interesa evaluar el desempeño de nuestro modelo en predecir los valores de una variable **Y** como función de las variables independientes **X** que definimos. Para esto existen dos funciones clásicas: 
* la raiz del error cuadrático medio (_RMSE_)
* el error absoluto medio (_MAE_)

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

### Linear 
X = housing_num_tr

housing_predictions = lin_reg.predict(X)
lin_mse = mean_squared_error(housing_value, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_mae = mean_absolute_error(housing_value, housing_predictions)

print("El MSE para el modelo lineal es: ",lin_rmse)
print("El MAE es el modelo lineal es: ",lin_mae)

In [ ]:
### Desicion Trees

housing_predictions = tree_reg.predict(data_full_prepared)
tree_mse = mean_squared_error(housing_value, housing_predictions)
tree_rmse = np.sqrt(tree_mse)
lin_mae = mean_absolute_error(housing_value, housing_predictions)

print("El MSE para el TreeRegressor es: ",tree_rmse)
print("El MAE para el TreeRegressor es: ",tree_rmse)
print("Efectivamente acá hay un overfitting, que ya habíamos visto en los plots.")

In [ ]:
### Random Forest 
housing_predictions = forest_reg.predict(data_full_prepared)
forest_mse = mean_squared_error(housing_value, housing_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_mae = mean_absolute_error(housing_value, housing_predictions)

print("El MSE para el RandomForest es: ",forest_rmse)
print("El MAE para el TreeRegressor es: ",forest_mae)

# CV

Para evaluar el desempeño de estos modelos sin utilizar nuestro set de test podemos hacer una nueva división de la partición de training. De la misma manera que antes, separamos una porción de los datos y luego entrenamos con el resto. Lo usual es dividir a los datos en 5 o 10 subconjuntos excluyentes entre sí y separar uno como el nuevo test. Con el resultado del entrenamiento aplicamos el ajuste a los datos separados y evaluamos el modelo con una función de score, por ej MSE. 
Para ver que los resultados no dependen de la partición utilizada volvemos a hacer este procedimiento utilizando los mismos 5 o 10 subconjuntos que habíamos definido, pero ahora elejimos otro para test. Haciendo esto para cada uno de los subconjunots tendremos 5, o 10, evaluaciones del modelo con su score.


In [ ]:
from sklearn.model_selection import cross_val_score

### definamos una función para presentar los resultados 
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())
    

## CV Linear

In [ ]:
housing_predictions = lin_reg.predict(X)
lin_scores = cross_val_score(lin_reg, X, housing_value, scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

## CV Trees

In [ ]:
housing_predictions = tree_reg.predict(data_full_prepared)
scores = cross_val_score(tree_reg, data_full_prepared, housing_value,scoring="neg_mean_squared_error", cv=10) ### hacemos una particion de 10
tree_rmse_scores = np.sqrt(-scores)
display_scores(tree_rmse_scores)

## CV Forest

In [ ]:
housing_predictions = forest_reg.predict(data_full_prepared)
scores = cross_val_score(forest_reg , data_full_prepared, housing_value,scoring="neg_mean_squared_error", cv=10) ### hacemos una particion de 10
tree_rmse_scores = np.sqrt(-scores)
display_scores(tree_rmse_scores)

## CONCLUSIONES

Del CV que realizamos sobre los tres algoritmos que propusimos resulta que el más prometedor es el RandomForest, dado que el valor medio de su error es sistemáticament más chico que el de la regresión lineal y los árboles de decisión. Más aún, tiene menos dispersión, con lo que podemos inferir que el ajuste es similar para cada "fold" (partición) del cross validation. 

# Fine Tuning

El resultado anterior nos dio como mejor candidato a los RandomForest como algoritmo para hacer nuestro ajuste. Tomando a éste como el algoritmo definitivo, podemos explorar los parámetros que definen a este algoritmo y ver si hay alguna combinación de ellos que mejoran aún más nuestro score. A los parámetros que definen al algoritmo se los suele llamar _hyperparameters_ para diferenciarlos de los otros parámetros ajustados al implementar un algoritmo. En otras palabras, los hiperparámetros son variables estructurales del algoritmo que definen de antemano cómo se va a hacer el fitteo de los datos. 

## GRIDSEARCH CV

El primer acercamiento a encontrar el mejor conjunto de hiperparámetros consiste en definir una grilla de valores a evaluar y comparar los scores medios del 10-fold CV hasta encontrar el mejor.

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    # try 12 (3×4) combinations of hyperparameters
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    # then try 6 (2×3) combinations with bootstrap set as False
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
  ]

forest_reg = RandomForestRegressor(random_state=42)
# train across 5 folds, that's a total of (12+6)*5=90 rounds of training 
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error', return_train_score=True)
grid_search.fit(data_full_prepared, housing_value)

### Resultados

In [ ]:
print("Los mejores parámetros son:",grid_search.best_params_)

In [ ]:
grid_search.best_estimator_

### Veamos los resultados de cada combinación de Hiperparámetros

In [ ]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

## RANDOM SEARCH CV

En este caso en lugar de definir una grilla, exploramos muestreos de los hiperparámetros en alguna distribución.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_distribs = {
        'n_estimators': randint(low=1, high=200),
        'max_features': randint(low=1, high=8),
    }

forest_reg = RandomForestRegressor(random_state=42)
rnd_search = RandomizedSearchCV(forest_reg, param_distributions=param_distribs,
                                n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=42)
rnd_search.fit(data_full_prepared, housing_value)

(OJO Tarda 2 minutos)

In [ ]:
cvres = rnd_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

In [ ]:
print(rnd_search.best_estimator_)

## Feature importances

En el caso de Desicion Trees o Random Forest, existe una métrica sobre nuestros features que determina cuáles son los más importantes en definir una categoría en el caso de clasificación o un valor de la variable dependiente en el caso de una regresión. Veamos cómo se rankean los Features en nuestro caso:

In [ ]:
feature_importances = grid_search.best_estimator_.feature_importances_

features = list(housing_num.columns) + list(cat_encoder.categories_[0])

In [ ]:
fig, ax = plt.subplots(figsize = (12,7))

# Example data
people = ('Tom', 'Dick', 'Harry', 'Slim', 'Jim')
y_pos = np.arange(len(people))
performance = 3 + 10 * np.random.rand(len(people))
error = np.random.rand(len(people))

ax.barh(features, feature_importances, align='center')
plt.yticks(fontsize = 14)
plt.xticks(fontsize = 14)
ax.set_xlabel('Feature Importance',fontsize = 16)
ax.grid()
#ax.set_title('How fast do you want to go today?')
#ax.set_ticklabels(["$%dk"%(round(v/1000)) for v in tick_values], fontsize=14)

#plt.show()

# Modelo final y evaluación en TEST

In [ ]:
### Defino el modelo final como el mejor entre grid y rnd
final_model = rnd_search.best_estimator_


In [ ]:
housing_test = pd.read_pickle("datasets/housing_test.pkl")
X_test = housing_test.drop("median_house_value", axis=1)
Y_test = housing_test["median_house_value"].copy()

## Hago las nuevas variables

X_test["rooms_per_household"] = X_test["total_rooms"]/X_test["households"]
X_test["bedrooms_per_room"] = X_test["total_bedrooms"]/X_test["total_rooms"]
X_test["population_per_household"]=X_test["population"]/X_test["households"]


### datos para la distancia.
p0 = (-124.55,38)
p1 = (-118.5, 32.5)
m = (p1[1]-p0[1])/(p1[0]-p0[0])
b = p0[1]-m*p0[0]
X_test["coast_distance"] =  abs(m*X_test["longitude"] - X_test["latitude"] + b)/np.sqrt(m**2+1)


### Relleno los NANS con medias

median_totbed = X_test["total_bedrooms"].median()
median_bedroom = X_test["bedrooms_per_room"].median()

### 

X_test["total_bedrooms"].fillna(median_totbed, inplace=True)
X_test["bedrooms_per_room"].fillna(median_bedroom, inplace=True)


In [ ]:
X_test.columns

In [ ]:
### transformacion NORMAL de  los X
test_num = X_test.drop(["ocean_proximity"],axis = 1) ### Trabajaemos con los datos numericos, dropeamos los categoricos
scaler = StandardScaler() ## Instancia de clase StandardScaler
scaler.fit(test_num) ### el scaler necesita saber el valor medio y desvio de cada variable, el método fit los obtiene
test_num_tr = scaler.transform(test_num)

test_cat = X_test[["ocean_proximity"]]
cat_encoder = OneHotEncoder()
test_cat_1hot = cat_encoder.fit_transform(test_cat)
test_cat_tr = test_cat_1hot.toarray()

In [ ]:
### juntamos los resultadsos numericos y los categoricos
data_test_prepared = np.concatenate((test_num_tr, test_cat_tr), axis=1)

In [ ]:

### tran

final_predictions = final_model.predict(data_test_prepared)

final_mse = mean_squared_error(Y_test, final_predictions)
final_rmse = np.sqrt(final_mse)

In [ ]:
final_rmse

**ESTE ES EL ERROR FINAL DE NUESTRO MODELO.**

# Exporto el modelo para producción

In [ ]:
with open("final_model.p","wb") as f:
    pickle.dump(final_model,f)